### Definition

Decorators provide a simple syntax for calling higher order functions.

Decorator is a function that takes another function and extends the behavior of the latter, without explicitly modifying it.

### Functions

Return a value based on given arguments.

Functions are **first class objects** i.e they can be passed around and used as arguments, like any other value(int,float etc.)

In [3]:
def foo(bar):
    return bar + 2

print foo
print foo(2)
print type(foo)

<function foo at 0x7f371b537ed8>
4
<type 'function'>
None


In [4]:
def call_foo_with_args(foo, arg):
    return foo(arg)
print call_foo_with_args(foo, 3)

5


### Nested Functions

In [17]:
def parent():
    
    print "Printing from parent method()"
    
    def first_child():
        return "Printing from first child"
        
    def second_child():
        return "Printing from second child"
    
    print first_child()
    print second_child()

In [18]:
parent()

Printing from parent method()
Printing from first child
Printing from second child


In [14]:
# first_child()
 
# Throws error because scope of first_child 
# method is limited to parent() method.
# Hence cannot be called from outside the method.

### Returning functions

In [20]:
def parent(num):
    
    print "Printing from parent method()"
    
    def first_child():
        return "Printing from first child"
        
    def second_child():
        return "Printing from second child"
    
    try:
        assert num==10
        return first_child
    
    except AssertionError:
        return second_child

In [23]:
foo = parent(10)
bar = parent(20)
print foo
print bar

Printing from parent method()
Printing from parent method()
<function first_child at 0x7f37147f4c80>
<function second_child at 0x7f37147f4e60>


In [22]:
foo()

'Printing from first child'

In [24]:
bar()

'Printing from second child'

### Example 1:

In [28]:
def myFunc(another_func):
    def wrapper():
        print "Before calling the function"

        another_func()

        print "After calling the function"
    return wrapper

In [29]:
def say_cheese():
    print "Say Cheeese!!"

In [33]:
some_func = myFunc(say_cheese)
some_func()

Before calling the function
Say Cheeese!!
After calling the function


In [39]:
from decorator1 import my_decorator

In [40]:
@my_decorator
def some_func():
    print "Say Cheese"

some_func()

Inside my_decorator function
Yes
Say Cheese
After calling the function


** Note : **
@my_decorator is just an easier way to say

**just_some_func = my_decorator(some_func)**

### Tracking function execution time in python

In [41]:
import time

In [67]:
def timing_function(any_func):
    
    def wrapper(*args, **kwargs):
        start = time.time()
        any_func(*args, **kwargs)
        end = time.time()
        print "Start time: " + str(start)
        print "End Time: " + str (end)
        return "Diff in time: " + str(end-start)
    return wrapper

In [45]:
num_list = []
@timing_function
def loop():
    for i in range(1000000):
        num_list.append(i)
    
loop()

Start time: 1504963595.83
End Time: 1504963595.93


'Diff in time: 0.093132019043'

In [46]:
@timing_function
def func_tools():
    return reduce(lambda x,y : x**y, [1,2,3,4,5])

In [48]:
some_func = func_tools()

Start time: 1504963755.08
End Time: 1504963755.08


In [49]:
some_func

'Diff in time: 4.05311584473e-06'

### Using Same Decorator for Multiple Functions

In [50]:
def decorator_func(original_function):
    
    def wrapper(*args, **kwargs):
        return original_function(*args, **kwargs)
    return wrapper

In [53]:
@decorator_func
def display():
    print "Hello from display"

print display
display()

<function wrapper at 0x7f371481c410>
Hello from display


In [55]:
@decorator_func
def display_info(name,age):
    print "My name is {0} and my age is {1}" .format(name,age)

print display_info
display_info('Rooney',30)

<function wrapper at 0x7f371481c6e0>
My name is Rooney and my age is 30


### Using Classes to Decorate Functions

In [81]:
class decorator_class(object):
    def __init__(self, original_func):
        self.original_func = original_func
        
    def __call__(self, *args, **kwargs):
        print "In Call Method"
        return self.original_func(*args, **kwargs)
@decorator_class
def display():
    print "hello"
display()

@decorator_class
def display_info(name,age):
    print "My name is {0} and my age is {1}" .format(name,age)

print display_info
display_info('Rooney',30)

In Call Method
hello
In Call Method
My name is Rooney and my age is 30


### Decorators for Logs

In [58]:
import logging
def my_logger(some_func):
    
    logging.basicConfig(filename = '{}.log'.format(some_func.__name__), level = logging.INFO)
    def wrapper(*args, **kwargs):
        logging.info('Ran with args: {} and kwargs: {}'.format(args,kwargs))
        return some_func(*args,**kwargs)
    return wrapper

In [59]:
@my_logger
def display():
    print "hello"
    
display()

hello


In [61]:
@my_logger
def display_info(name,age):
    print "Name: {0} and Age: {1}".format(name,age)
    
display_info('Rooney',30)

Name: Rooney and Age: 30


## Chaining Multiple Decorators

In [73]:
@timing_function
@my_logger

def display_info(name):
    print name
    
#some_func = timing_function(my_logger(display_info))
print(display_info.__name__)
timer('manutd')

wrapper
manutd
Start time: 1504969181.3
End Time: 1504969181.3


'Diff in time: 0.000271081924438'

The above chained decorator call is equivalent to

** some_func = timing_function(my_logger(display_info)) **

** Important ** : 

- We want to apply display_info to both the decorators.
- But what's happening is first my_logger(display_info) get executed, which returns            wrapper function.
- This **WRAPPER** function is decorated with timing function, instead of display_info function.
- To fix this issue, we use wraps function from functools module.

##### Fix 

In [74]:
from functools import wraps

In [75]:
def timing_function(any_func):
    @wraps(any_func)
    def wrapper(*args, **kwargs):
        start = time.time()
        any_func(*args, **kwargs)
        end = time.time()
        print "Start time: " + str(start)
        print "End Time: " + str (end)
        return "Diff in time: " + str(end-start)
    return wrapper

In [77]:
import logging
def my_logger(some_func):
    logging.basicConfig(filename = '{}.log'.format(some_func.__name__), level = logging.INFO)

    @wraps(some_func)
    def wrapper(*args, **kwargs):
        logging.info('Ran with args: {} and kwargs: {}'.format(args,kwargs))
        return some_func(*args,**kwargs)
    return wrapper

In [80]:
# @timing_function
# @my_logger

def display_info(name,age):
    print "Name: {0}, Age: {1} ".format(name, age)
    
#some_func = timing_function(my_logger(display_info))
display_info('Marcus Rashford', 19)

some_func = my_logger(display_info)
print timing_function(some_func)

Name: Marcus Rashford, Age: 19 
<function display_info at 0x7f371481cf50>
